# RAG ChatBot Example

This file uses the OpenAI LLM for text generation and hence it requires an API key. You can acquire this API key from [OpenAI's website](https://openai.com).

### Get the API key from the user

In [8]:
from getpass import getpass

OPENAI_API_KEY = getpass("Enter your api key for OpenAI: ")

### Load Extracted data from file generated by running `extract_data.py`

In [9]:
import json

with open("./data/extracted.json", "r") as file:
    data = json.load(file)

### Create an object to create embedding from documents and queries

In [10]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

### Create a Vector Store to save documents generated for searching relevant answers later

In [11]:
from langchain.vectorstores.chroma import Chroma

texts = []
metadatas = []
for key, value in data.items():
    metadatas.append({"heading": key})
    texts.append(value)
retriever = Chroma.from_texts(
    texts=texts, embedding=embeddings, metadatas=metadatas
).as_retriever(search_kwargs={"k": 3})

### Create an object that connects to the LLM and can be used to query later

In [12]:
from langchain.llms import OpenAI

llm = OpenAI(openai_api_key=OPENAI_API_KEY)

### Create a chain that ties all the parts together

In [13]:
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_llm(llm=llm, retriever=retriever)

### Start querying the bot

In [14]:
chain.run({"query": "How is SPI calculated?"})

' SPI is the weighted average of the grade points obtained in all the courses registered by the student during the semester. For example, if a student passes five courses (Theory/labs. /Projects/ 30 Seminar etc.) in a semester with credits C1, C2, C3, C4 and C5 and her/his grade points in these courses are g1, g2, g3, g4 and g5 respectively, then her/his SPI is equal to: \nC1g1 + C2 g2 + C3 g3 + C4 g4 + C5 g5 \nSPI =  ---------------------------------------------------------- \nC1 + C2 + C3 + C4 + C5'

In [15]:
chain.run({"query": "What is the minimum criteria required to complete a btech project?"})

' A student is required to do an innovative work with the application of knowledge gained from courses in the earlier years under the guidance of a faculty member in the sixth semester or later.'

In [16]:
chain.run({"query": "What is the IDDDP programme?"})

' The IDDDP programme is an interdisciplinary dual degree programme at IIT Bombay that allows undergraduate students admitted to B.S., B.Tech. and DD (B.Tech.+M.Tech.) programmes to apply for all the specializations of Dual Degree (DD) and M.Tech. programmes approved by the Academic Senate of IIT Bombay. Students must have a CPI of 7.5 or higher and should not have any FR/DR/DX/W grades in mandatory courses, including NSO/NSS/NCC, and must apply by 15th July of the calendar year.'